In [1]:
from utils import load_data
dataset = load_data('../data/dev-v1.1-processed.json')

Load 10570 examples from ../data/dev-v1.1-processed.json...


In [2]:
data = dataset[55: 85]
data

In [3]:
def get_word(passage, start, end):
    from preprocess import SpacyTokenizer
    mine = SpacyTokenizer()
    tokenized = [i[0] for i in mine.tokenize(passage).data]
    joined = ' '. join(tokenized[start : end + 1])
    return joined

In [4]:
print('Question:', data[0].question)
print('Answer:', get_word(data[0].passage, data[0].y_start.index(1), data[0].y_end.index(1)))

Question: Who was the Super Bowl 50 MVP ?
Answer: Von Miller


In [5]:
class custom_arguments:
    def __init__(self, args):
        self.batch_size = args['batch_size']
        self.doc_layers = args['doc_layers']
        self.dropout_emb = args['dropout_emb']
        self.dropout_rnn_output = args['dropout_rnn_output']
        self.embedding_file = args['embedding_file']
        self.epoch = args['epoch']
        self.finetune_topk = args['finetune_topk']
        self.gpu = args['gpu']
        self.grad_clipping = args['grad_clipping']
        self.hidden_size = args['hidden_size']
        self.lr = args['lr']
        self.ner_emb_dim = args['ner_emb_dim']
        self.optimizer = args['optimizer']
        self.pos_emb_dim = args['pos_emb_dim']
        self.pretrained = args['pretrained']
        self.rel_emb_dim = args['rel_emb_dim']
        self.rnn_padding = args['rnn_padding']
        self.rnn_type = args['rnn_type']
        self.seed = args['seed']
        self.test_mode = args['test_mode']
        self.use_cuda = args['use_cuda']
        self.p_max_size = args['p_max_size']
        
        
args_dict = {'batch_size':32, 'doc_layers':1, 'dropout_emb':0.4, 'dropout_rnn_output':0.4,
             'embedding_file':'/home/unchartech001/Local_Resources/glove.6B/glove.6B.300d.txt',
             'epoch':50, 'finetune_topk':10, 'gpu':'0', 'grad_clipping':10.0, 'hidden_size':96,
             'lr':0.002, 'ner_emb_dim':8, 'optimizer':'adamax', 'pos_emb_dim':12, 'pretrained':'',
             'rel_emb_dim':10, 'rnn_padding':True, 'rnn_type':'lstm', 'seed':1234, 'test_mode':False,
             'use_cuda':True, 'p_max_size':750}
args = custom_arguments(args_dict)

In [6]:
import model
from utils import build_vocab
build_vocab()
model = model.Model(args)

Load vocabulary from ../data/vocab...
Vocabulary size: 33382
Load pos vocabulary from ../data/pos_vocab...
POS vocabulary size: 51
Load ner vocabulary from ../data/ner_vocab...
NER vocabulary size: 20
Load relation vocabulary from ../data/rel_vocab...
Rel vocabulary size: 39
Use cuda: False
TriAN(
  (embedding): Embedding(33382, 300, padding_idx=0)
  (pos_embedding): Embedding(51, 12, padding_idx=0)
  (ner_embedding): Embedding(20, 8, padding_idx=0)
  (rel_embedding): Embedding(39, 10, padding_idx=0)
  (p_q_emb_match): SeqAttnMatch(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
  (doc_rnn): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(635, 96, bidirectional=True)
    )
  )
  (question_rnn): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(312, 96, bidirectional=True)
    )
  )
  (q_self_attn_start): LinearSeqAttn(
    (linear): Linear(in_features=192, out_features=1, bias=True)
  )
  (p_q_attn_start): BilinearSeqAttn(
    (linear): Linear(in_feature

In [11]:
prediction = model.predict(dataset)
len(prediction)

KeyboardInterrupt: 

In [16]:
prediction [0].size()

torch.Size([32, 750])

In [8]:
train_ = model.train(dataset)

ValueError: Expected input batch_size (32) to match target batch_size (149).

In [ ]:
import torch
def output_to_map(prediction):
    start_maps, end_maps = [], []
    for sub_start, sub_end in zip(prediction[0], prediction[1]):
        start = int(torch.argmax(sub_start))
        end = int(torch.argmax(sub_end))
        start_maps.append([0] * start + [1] + [0]*(len(prediction[0]) - start))
        end_maps.append([0] * end + [1] + [0]*(len(prediction[0]) - end))
    return start_maps, end_maps

In [ ]:
def map_padding(start, end):
    padded = []
    padded.extend([0] * start.index(1))
    padded.extend([1] * (end.index(1) - start.index(1) + 1))
    padded.extend([0] * (len(start) - end.index(1) - 1))
    return padded

In [ ]:
def evaluate(model, dev_data):
    from sklearn.metrics import f1_score
    f1_scores = []
    for batch_input in model._iter_data(dev_data):
            print(batch_input)
            feed_input = [x for x in batch_input[:-1]]
            pred_proba = model.network(*feed_input)
            print(pred_proba)
            map_pred = output_to_map(pred_proba)
            for i, data in enumerate(feed_input):
                print(data)
                truth = map_padding(data.y_start, data.y_end)
                prediction = map_padding(map_pred[0][i], map_pred[1][i])
                f1_scores += f1_score(truth, prediction)
    return sum(f1_scores)/len(f1_scores)

In [ ]:
import torch

x_in, y_in = 32, 750
x_lin = x_in
y_lin = 750


m = torch.nn.Linear(x_lin, y_lin)

i1 = torch.randn(x_in, y_in)
print(i1.size())
i1_mask = torch.zeros(x_in, y_in).byte()
i2 = torch.randn(x_in, y_in)


Wi2 = m(torch.transpose(i2, 0, 1))

print(Wi2.size())
i1Wi2 = i1.mm(Wi2)
print(i1Wi2.size())


In [ ]:
i1Wi2 - i1Wi2.data.masked_fill_(i1_mask.data, -float('inf'))

In [ ]:
batch = torch.randn(30, 750)

In [ ]:
a = [1,2]

In [ ]:
a += 4*[0]

In [ ]:
a

In [ ]:
a[-1]